<a href="https://colab.research.google.com/github/dhruszecki/cdatos-st-tp/blob/main/postprocesado_parametros_st_tp_turing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

import os
import os.path
from os import path

import matplotlib.pyplot as plt

from enum import Enum
from time import time


# para ejecutar desde google colab, con notebook en el directorio donde están los demás archivos
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Utils

In [ ]:
def download_dataset_if_needed(filename):
    if not path.exists("dataset/"+filename):
        url = "https://raw.githubusercontent.com/dhruszecki/cdatos-ST-TP/main/data/"+filename
        !wget {url} -P dataset
    else:
        print("dataset/" + filename, " already exists! :)")

In [ ]:
download_dataset_if_needed("df-filtered.csv")

--2021-10-04 01:55:37--  https://raw.githubusercontent.com/dhruszecki/cdatos-ST-TP/main/data/df-filtered.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4969468 (4.7M) [text/plain]
Saving to: ‘dataset/df-filtered.csv’

df-filtered.csv     100%[===================>]   4.74M  --.-KB/s    in 0.08s   

2021-10-04 01:55:37 (59.6 MB/s) - ‘dataset/df-filtered.csv’ saved [4969468/4969468]



In [ ]:
df = pd.read_csv("dataset/df-filtered.csv")
df.head()

,Unnamed: 0,id,sender,text,sender_class,dialog_id,filtered_tokens_col
0,0,0,participant2,I love iphone! i just bought new iphone!,Bot,0xab38710,"['love', 'iphone', '!', 'bought', 'new', 'ipho..."
1,1,1,participant1,"Thats good for you, i'm not very into new tech",Human,0xab38710,"['s', 'good', ',', 'new', 'tech']"
2,2,2,participant2,I am a college student and i am a college student,Bot,0xab38710,"['college', 'student', 'college', 'student']"
3,3,3,participant1,I am go to gym and live on donations,Human,0xab38710,"['gym', 'live', 'donations']"
4,4,4,participant2,I am a vegan and i am in the midwest,Bot,0xab38710,"['vegan', 'midwest']"


In [ ]:
#Obtengo el id de todos los dialogos junto con el indice de inicio en el dataframe
dialogos=df.drop_duplicates(subset = ["dialog_id"])
dialogos=dialogos['dialog_id']
dialogos= pd.DataFrame(dialogos)

In [ ]:
dialogos

,dialog_id
0,0xab38710
11,0x223f6d5e
13,0x6789b32a
15,0x7304a92a
16,0x7470f02a
...,...
47199,0xd58490a
47206,0x5c877561
47207,0x3dcfa9b9
47208,0x44c1894b


In [ ]:
dialogos['diccionario_total']='diccionario_total'
dialogos['diccionario_participant1']='diccionario_participant1'
dialogos['diccionario_participant2']='diccionario_participant2'
dialogos['diccionario_interseccion']='diccionario_interseccion'

In [ ]:
dialogos

,dialog_id,diccionario_total,diccionario_participant1,diccionario_participant2,diccionario_interseccion
0,0xab38710,diccionario_total,diccionario_participant1,diccionario_participant2,diccionario_interseccion
11,0x223f6d5e,diccionario_total,diccionario_participant1,diccionario_participant2,diccionario_interseccion
13,0x6789b32a,diccionario_total,diccionario_participant1,diccionario_participant2,diccionario_interseccion
15,0x7304a92a,diccionario_total,diccionario_participant1,diccionario_participant2,diccionario_interseccion
16,0x7470f02a,diccionario_total,diccionario_participant1,diccionario_participant2,diccionario_interseccion
...,...,...,...,...,...
47199,0xd58490a,diccionario_total,diccionario_participant1,diccionario_participant2,diccionario_interseccion
47206,0x5c877561,diccionario_total,diccionario_participant1,diccionario_participant2,diccionario_interseccion
47207,0x3dcfa9b9,diccionario_total,diccionario_participant1,diccionario_participant2,diccionario_interseccion
47208,0x44c1894b,diccionario_total,diccionario_participant1,diccionario_participant2,diccionario_interseccion


In [ ]:
from time import sleep

for index, row in dialogos.iterrows():
  temp_dialog=df[df['dialog_id']==row['dialog_id']]
  diccionarioTotal = []

  for index_temp, row_temp in temp_dialog.iterrows():
      non_stop_words=row_temp['filtered_tokens_col'].replace(" ", "").replace("'", "").strip('][').split(",")
      #print ("Non_stop_words:",non_stop_words)
      diccionarioTotal.extend(non_stop_words)
      #print ("diccionarioTotal pre filtro:",diccionarioTotal)
      diccionarioTotal=list(dict.fromkeys(diccionarioTotal))
      #print ("diccionarioTotal drop:",diccionarioTotal)

  str1 = "','"
  tempjoin=str1.join(diccionarioTotal)  
  dialogos.loc[dialogos['dialog_id']==row['dialog_id'], 'diccionario_total']=tempjoin

  #diccionario participant1#
  temp_dialog=df[(df['dialog_id']==row['dialog_id'])&(df['sender']=='participant1')]
  diccionarioParticipant1 = []

  for index_temp, row_temp in temp_dialog.iterrows():
      non_stop_words=row_temp['filtered_tokens_col'].replace(" ", "").replace("'", "").strip('][').split(",")
      diccionarioParticipant1.extend(non_stop_words)
      diccionarioParticipant1=list(dict.fromkeys(diccionarioParticipant1))
  str1 = "','"
  tempjoin1=str1.join(diccionarioParticipant1)  
  dialogos.loc[dialogos['dialog_id']==row['dialog_id'], 'diccionario_participant1']=tempjoin1

  #diccionario participant2#
  temp_dialog=df[(df['dialog_id']==row['dialog_id'])&(df['sender']=='participant2')]
  diccionarioParticipant2 = []

  for index_temp, row_temp in temp_dialog.iterrows():
      non_stop_words=row_temp['filtered_tokens_col'].replace(" ", "").replace("'", "").strip('][').split(",")
      diccionarioParticipant2.extend(non_stop_words)
      diccionarioParticipant2=list(dict.fromkeys(diccionarioParticipant2))
  str1 = "','"
  tempjoin2=str1.join(diccionarioParticipant2)  
  dialogos.loc[dialogos['dialog_id']==row['dialog_id'], 'diccionario_participant2']=tempjoin2

  ##diccionario_interseccion##
  diccionario_interseccion = list(set(diccionarioParticipant1) & set(diccionarioParticipant2))
  str1 = "','"
  tempjoin3=str1.join(diccionario_interseccion)  
  dialogos.loc[dialogos['dialog_id']==row['dialog_id'], 'diccionario_interseccion']=tempjoin3



KeyboardInterrupt: ignored

In [ ]:
pd.set_option("max_colwidth", None)
pd.set_option("display.max_rows", None)
dialogos.head()

,dialog_id,diccionario_total,diccionario_participant1,diccionario_participant2,diccionario_interseccion
0,0xab38710,"love','iphone','!','bought','new','s','good','','tech','college','student','gym','live','donations','vegan','midwest','...','dogs','maybe','told','eat','cheap','salads','insted','meat','mind','having','office','hard','Dogs','?','vegetarian','Strange','answer','guess','know','ps4','experience','.","s','good','','new','tech','gym','live','donations','vegan','...','dogs','maybe','told','eat','cheap','salads','insted','meat','Dogs','office','?','Strange','answer","love','iphone','!','bought','new','college','student','vegan','midwest','mind','having','office','hard','vegetarian','guess','know','ps4','experience','.","vegan','office','new"
11,0x223f6d5e,"Hello','/test","Hello','/test",,
13,0x6789b32a,"Hey','?","Hey','?",,
15,0x7304a92a,"love','painting','','paint",,"love','painting','','paint",
16,0x7470f02a,Hello,Hello,,


In [ ]:
#merge con dataset de dialogos
df_diccs = pd.merge(df, dialogos, on="dialog_id")

In [ ]:
pd.set_option("max_colwidth", None)
pd.set_option("display.max_rows", None)

df_diccs.head(100)

,Unnamed: 0,id,sender,text,sender_class,dialog_id,filtered_tokens_col,diccionario_total,diccionario_participant1,diccionario_participant2,diccionario_interseccion
0,0,0,participant2,I love iphone! i just bought new iphone!,Bot,0xab38710,"['love', 'iphone', '!', 'bought', 'new', 'iphone', '!']","love','iphone','!','bought','new','s','good','','tech','college','student','gym','live','donations','vegan','midwest','...','dogs','maybe','told','eat','cheap','salads','insted','meat','mind','having','office','hard','Dogs','?','vegetarian','Strange','answer','guess','know','ps4','experience','.","s','good','','new','tech','gym','live','donations','vegan','...','dogs','maybe','told','eat','cheap','salads','insted','meat','Dogs','office','?','Strange','answer","love','iphone','!','bought','new','college','student','vegan','midwest','mind','having','office','hard','vegetarian','guess','know','ps4','experience','.","vegan','office','new"
1,1,1,participant1,"Thats good for you, i'm not very into new tech",Human,0xab38710,"['s', 'good', ',', 'new', 'tech']","love','iphone','!','bought','new','s','good','','tech','college','student','gym','live','donations','vegan','midwest','...','dogs','maybe','told','eat','cheap','salads','insted','meat','mind','having','office','hard','Dogs','?','vegetarian','Strange','answer','guess','know','ps4','experience','.","s','good','','new','tech','gym','live','donations','vegan','...','dogs','maybe','told','eat','cheap','salads','insted','meat','Dogs','office','?','Strange','answer","love','iphone','!','bought','new','college','student','vegan','midwest','mind','having','office','hard','vegetarian','guess','know','ps4','experience','.","vegan','office','new"
2,2,2,participant2,I am a college student and i am a college student,Bot,0xab38710,"['college', 'student', 'college', 'student']","love','iphone','!','bought','new','s','good','','tech','college','student','gym','live','donations','vegan','midwest','...','dogs','maybe','told','eat','cheap','salads','insted','meat','mind','having','office','hard','Dogs','?','vegetarian','Strange','answer','guess','know','ps4','experience','.","s','good','','new','tech','gym','live','donations','vegan','...','dogs','maybe','told','eat','cheap','salads','insted','meat','Dogs','office','?','Strange','answer","love','iphone','!','bought','new','college','student','vegan','midwest','mind','having','office','hard','vegetarian','guess','know','ps4','experience','.","vegan','office','new"
3,3,3,participant1,I am go to gym and live on donations,Human,0xab38710,"['gym', 'live', 'donations']","love','iphone','!','bought','new','s','good','','tech','college','student','gym','live','donations','vegan','midwest','...','dogs','maybe','told','eat','cheap','salads','insted','meat','mind','having','office','hard','Dogs','?','vegetarian','Strange','answer','guess','know','ps4','experience','.","s','good','','new','tech','gym','live','donations','vegan','...','dogs','maybe','told','eat','cheap','salads','insted','meat','Dogs','office','?','Strange','answer","love','iphone','!','bought','new','college','student','vegan','midwest','mind','having','office','hard','vegetarian','guess','know','ps4','experience','.","vegan','office','new"
4,4,4,participant2,I am a vegan and i am in the midwest,Bot,0xab38710,"['vegan', 'midwest']","love','iphone','!','bought','new','s','good','','tech','college','student','gym','live','donations','vegan','midwest','...','dogs','maybe','told','eat','cheap','salads','insted','meat','mind','having','office','hard','Dogs','?','vegetarian','Strange','answer','guess','know','ps4','experience','.","s','good','','new','tech','gym','live','donations','vegan','...','dogs','maybe','told','eat','cheap','salads','insted','meat','Dogs','office','?','Strange','answer","love','iphone','!','bought','new','college','student','vegan','midwest','mind','having','office','hard','vegetarian','guess','know','ps4','experience','.","vegan','office','new"
5,5,5,participant1,So vegan... i have d

In [ ]:
df_diccs.columns

Index(['Unnamed: 0', 'id', 'sender', 'text', 'sender_class', 'dialog_id',
       'filtered_tokens_col', 'diccionario_total', 'diccionario_participant1',
       'diccionario_participant2', 'diccionario_interseccion'],
      dtype='object')

In [ ]:
df_diccs_parametros=df_diccs

In [ ]:
import re

for index, row_temp in df_diccs_parametros.iterrows():

  list_key_words=row_temp['filtered_tokens_col'].replace(" ", "").replace("'", "").strip('][').split(",")
  list_diccionario_total=row_temp['diccionario_total'].replace(" ", "").replace("'", "").strip('][').split(",")
  list_diccionario_participant1=row_temp['diccionario_participant1'].replace(" ", "").replace("'", "").strip('][').split(",")
  list_diccionario_participant2=row_temp['diccionario_participant2'].replace(" ", "").replace("'", "").strip('][').split(",")
  list_diccionario_interseccion=row_temp['diccionario_interseccion'].replace(" ", "").replace("'", "").strip('][').split(",")
    
    
  cant_key_words=len(list_key_words)
  cant_diccionario_total=len(list_diccionario_total)
  cant_diccionario_participant1=len(list_diccionario_participant1)
  cant_diccionario_participant2=len(list_diccionario_participant2)
  cant_diccionario_interseccion=len(list_diccionario_interseccion)
    
  ##dependientes de cada interlocucion
  if cant_diccionario_total > 0:
      keyWords_over_totalDic=cant_key_words/cant_diccionario_total
  else:
      keyWords_over_totalDic=0
    	
  if row_temp['sender'] == 'participant1':
    ##dependientes de cada interlocucion
    if cant_diccionario_participant1 > 0:
      keyWords_over_senderDic=cant_key_words/cant_diccionario_participant1
    else:
      keyWords_over_senderDic=0
    	
    if cant_diccionario_participant2 > 0:
      keyWords_over_listenerDic=len(list(set(list_key_words) & set(list_diccionario_participant2)))/cant_diccionario_participant2
    else:
      keyWords_over_listenerDic=0
	    
	  ##globales de todo el dialogo	    	
    if cant_diccionario_participant1 > 0:
      intersect_over_senderDic=cant_diccionario_interseccion/cant_diccionario_participant1
    else:
      intersect_over_senderDic=0
    	
    if cant_diccionario_total > 0:
      participantdicc_over_totaldicc=cant_diccionario_participant1/cant_diccionario_total
    else:
      participantdicc_over_totaldicc=0
  else:
    	##dependientes de cada interlocucion
    if cant_diccionario_participant2 > 0:
      keyWords_over_senderDic=cant_key_words/cant_diccionario_participant2
    else:
      keyWords_over_senderDic=0
    	
    if cant_diccionario_participant1 > 0:
      keyWords_over_listenerDic=len(list(set(list_key_words) & set(list_diccionario_participant1)))/cant_diccionario_participant1
    else:
      keyWords_over_listenerDic=0
	    
	  ##globales de todo el dialogo	    	
    if cant_diccionario_participant2 > 0:
      intersect_over_senderDic=cant_diccionario_interseccion/cant_diccionario_participant2
    else:
      intersect_over_senderDic=0
    	
    if cant_diccionario_total > 0:
      participantdicc_over_totaldicc=cant_diccionario_participant2/cant_diccionario_total
    else:
      participantdicc_over_totaldicc=0
    
  df_diccs_parametros.loc[(df_diccs_parametros['dialog_id']==row_temp['dialog_id']) & (df_diccs_parametros['id']==row_temp['id']), 'cant_key_words']=cant_key_words
  df_diccs_parametros.loc[(df_diccs_parametros['dialog_id']==row_temp['dialog_id']) & (df_diccs_parametros['id']==row_temp['id']), 'cant_diccionario_total']=cant_diccionario_total
  df_diccs_parametros.loc[(df_diccs_parametros['dialog_id']==row_temp['dialog_id']) & (df_diccs_parametros['id']==row_temp['id']), 'cant_diccionario_participant1']=cant_diccionario_participant1
  df_diccs_parametros.loc[(df_diccs_parametros['dialog_id']==row_temp['dialog_id']) & (df_diccs_parametros['id']==row_temp['id']), 'cant_diccionario_participant2']=cant_diccionario_participant2
  df_diccs_parametros.loc[(df_diccs_parametros['dialog_id']==row_temp['dialog_id']) & (df_diccs_parametros['id']==row_temp['id']), 'cant_diccionario_interseccion']=cant_diccionario_interseccion
  df_diccs_parametros.loc[(df_diccs_parametros['dialog_id']==row_temp['dialog_id']) & (df_diccs_parametros['id']==row_temp['id']), 'keyWords_over_totalDic']=keyWords_over_totalDic
  df_diccs_parametros.loc[(df_diccs_parametros['dialog_id']==row_temp['dialog_id']) & (df_diccs_parametros['id']==row_temp['id']), 'keyWords_over_senderDic']=keyWords_over_senderDic
  df_diccs_parametros.loc[(df_diccs_parametros['dialog_id']==row_temp['dialog_id']) & (df_diccs_parametros['id']==row_temp['id']), 'keyWords_over_listenerDic']=keyWords_over_listenerDic
  df_diccs_parametros.loc[(df_diccs_parametros['dialog_id']==row_temp['dialog_id']) & (df_diccs_parametros['id']==row_temp['id']), 'intersect_over_senderDic']=intersect_over_senderDic
  df_diccs_parametros.loc[(df_diccs_parametros['dialog_id']==row_temp['dialog_id']) & (df_diccs_parametros['id']==row_temp['id']), 'participantdicc_over_totaldicc']=participantdicc_over_totaldicc
  ###calculo de parametros adicionales
  interaccion=row_temp['text']
  word_list = interaccion.split()
  cant_caracteres=len(interaccion)
  cant_palabras_interaccion=len(word_list)
  cant_palabras_unicas=len(list(dict.fromkeys(word_list)))
  cant_puntuacion=len(re.sub('[\w]+' ,'', interaccion))
  df_diccs_parametros.loc[(df_diccs_parametros['dialog_id']==row_temp['dialog_id']) & (df_diccs_parametros['id']==row_temp['id']), 'cant_caracteres']=cant_caracteres
  df_diccs_parametros.loc[(df_diccs_parametros['dialog_id']==row_temp['dialog_id']) & (df_diccs_parametros['id']==row_temp['id']), 'cant_palabras_interaccion']=cant_palabras_interaccion
  df_diccs_parametros.loc[(df_diccs_parametros['dialog_id']==row_temp['dialog_id']) & (df_diccs_parametros['id']==row_temp['id']), 'cant_palabras_unicas']=cant_palabras_unicas
  df_diccs_parametros.loc[(df_diccs_parametros['dialog_id']==row_temp['dialog_id']) & (df_diccs_parametros['id']==row_temp['id']), 'cant_puntuacion']=cant_puntuacion
  ###calculo de uniqWeight (lo dejo comentado porque no funciono bien, es preferible calcularlo afuera mas adelante para experimentar)
  #if cant_diccionario_total > 0 & cant_palabras_interaccion > 0 & cant_caracteres > 0:
  #  uniqWeight=keyWords_over_listenerDic+((cant_diccionario_interseccion/cant_diccionario_total)*(keyWords_over_totalDic + keyWords_over_senderDic + keyWords_over_listenerDic + (cant_palabras_interaccion + cant_palabras_unicas + cant_palabras_interaccion)/(3*cant_palabras_interaccion) + (cant_puntuacion/cant_caracteres)))
  #else:
  #  uniqWeight=0
  #df_diccs_parametros.loc[(df_diccs_parametros['dialog_id']==row_temp['dialog_id']) & (df_diccs_parametros['id']==row_temp['id']), 'uniqWeight']=uniqWeight


In [ ]:
df_diccs_parametros.head(100)

,Unnamed: 0,id,sender,text,sender_class,dialog_id,filtered_tokens_col,diccionario_total,diccionario_participant1,diccionario_participant2,diccionario_interseccion,cant_key_words,cant_diccionario_total,cant_diccionario_participant1,cant_diccionario_participant2,cant_diccionario_interseccion,keyWords_over_totalDic,keyWords_over_senderDic,keyWords_over_listenerDic,intersect_over_senderDic,participantdicc_over_totaldicc,cant_caracteres,cant_palabras_interaccion,cant_palabras_unicas,cant_puntuacion,uniqWeight
0,0,0,participant2,I love iphone! i just bought new iphone!,Bot,0xab38710,"['love', 'iphone', '!', 'bought', 'new', 'iphone', '!']","love','iphone','!','bought','new','s','good','','tech','college','student','gym','live','donations','vegan','midwest','...','dogs','maybe','told','eat','cheap','salads','insted','meat','mind','having','office','hard','Dogs','?','vegetarian','Strange','answer','guess','know','ps4','experience','.","s','good','','new','tech','gym','live','donations','vegan','...','dogs','maybe','told','eat','cheap','salads','insted','meat','Dogs','office','?','Strange','answer","love','iphone','!','bought','new','college','student','vegan','midwest','mind','having','office','hard','vegetarian','guess','know','ps4','experience','.","vegan','office','new",7.0,39.0,23.0,19.0,3.0,0.179487,0.368421,0.043478,0.157895,0.487179,40.0,8.0,7.0,9.0,0.0
1,1,1,participant1,"Thats good for you, i'm not very into new tech",Human,0xab38710,"['s', 'good', ',', 'new', 'tech']","love','iphone','!','bought','new','s','good','','tech','college','student','gym','live','donations','vegan','midwest','...','dogs','maybe','told','eat','cheap','salads','insted','meat','mind','having','office','hard','Dogs','?','vegetarian','Strange','answer','guess','know','ps4','experience','.","s','good','','new','tech','gym','live','donations','vegan','...','dogs','maybe','told','eat','cheap','salads','insted','meat','Dogs','office','?','Strange','answer","love','iphone','!','bought','new','college','student','vegan','midwest','mind','having','office','hard','vegetarian','guess','know','ps4','experience','.","vegan','office','new",6.0,39.0,23.0,19.0,3.0,0.153846,0.260870,0.052632,0.130435,0.589744,46.0,10.0,10.0,11.0,0.0
2,2,2,participant2,I am a college student and i am a college student,Bot,0xab38710,"['college', 'student', 'college', 'student']","love','iphone','!','bought','new','s','good','','tech','college','student','gym','live','donations','vegan','midwest','...','dogs','maybe','told','eat','cheap','salads','insted','meat','mind','having','office','hard','Dogs','?','vegetarian','Strange','answer','guess','know','ps4','experience','.","s','good','','new','tech','gym','live','donations','vegan','...','dogs','maybe','told','eat','cheap','salads','insted','meat','Dogs','office','?','Strange','answer","love','iphone','!','bought','new','college','student','vegan','midwest','mind','having','office','hard','vegetarian','guess','know','ps4','experience','.","vegan','office','new",4.0,39.0,23.0,19.0,3.0,0.102564,0.210526,0.000000,0.157895,0.487179,49.0,11.0,7.0,10.0,0.0
3,3,3,participant1,I am go to gym and live on donations,Human,0xab38710,"['gym', 'live', 'donations']","love','iphone','!','bought','new','s','good','','tech','college','student','gym','live','donations','vegan','midwest','...','dogs','maybe','told','eat','cheap','salads','insted','meat','mind','having','office','hard','Dogs','?','vegetarian','Strange','answer','guess','know','ps4','experience','.","s','good','','new','tech','gym','live','donations','vegan','...','dogs','maybe','told','eat','cheap','salads','insted','meat','Dogs','office','?','Strange','answer","love','iphone','!','bought','new','college','student','vegan','midwest','mind','having','office','hard','vegetarian','guess','know','ps4','experience','.","vegan','office','new",3.0,39.0,23.0,19.0,3.0,0.076923,0.130435,0.000000,0.130435,0.589744,36.0,9.0,9.0,8.0,0.0
4,4,4,participant2,I am a vegan and i am in the midwest,Bot,0xab3871

In [ ]:
df_diccs_parametros_uniqWeight=df_diccs_parametros

In [ ]:
#aca podemos cambiar el uniqWeight
df_diccs_parametros_uniqWeight['uniqWeight']=df_diccs_parametros_uniqWeight.apply(lambda row: row.keyWords_over_listenerDic+((row.cant_diccionario_interseccion/row.cant_diccionario_total)*(0.25*row.keyWords_over_totalDic + 0.25*row.keyWords_over_senderDic + 2*row.keyWords_over_listenerDic + (row.cant_palabras_interaccion + row.cant_palabras_unicas + row.cant_palabras_interaccion)/(3*row.cant_palabras_interaccion) + (row.cant_puntuacion/row.cant_caracteres))), axis=1)

In [ ]:
df_diccs_parametros_uniqWeight.head(500)

,Unnamed: 0,id,sender,text,sender_class,dialog_id,filtered_tokens_col,diccionario_total,diccionario_participant1,diccionario_participant2,diccionario_interseccion,cant_key_words,cant_diccionario_total,cant_diccionario_participant1,cant_diccionario_participant2,cant_diccionario_interseccion,keyWords_over_totalDic,keyWords_over_senderDic,keyWords_over_listenerDic,intersect_over_senderDic,participantdicc_over_totaldicc,cant_caracteres,cant_palabras_interaccion,cant_palabras_unicas,cant_puntuacion,uniqWeight
0,0,0,participant2,I love iphone! i just bought new iphone!,Bot,0xab38710,"['love', 'iphone', '!', 'bought', 'new', 'iphone', '!']","love','iphone','!','bought','new','s','good','','tech','college','student','gym','live','donations','vegan','midwest','...','dogs','maybe','told','eat','cheap','salads','insted','meat','mind','having','office','hard','Dogs','?','vegetarian','Strange','answer','guess','know','ps4','experience','.","s','good','','new','tech','gym','live','donations','vegan','...','dogs','maybe','told','eat','cheap','salads','insted','meat','Dogs','office','?','Strange','answer","love','iphone','!','bought','new','college','student','vegan','midwest','mind','having','office','hard','vegetarian','guess','know','ps4','experience','.","vegan','office','new",7.0,39.0,23.0,19.0,3.0,0.179487,0.368421,0.043478,0.157895,0.487179,40.0,8.0,7.0,9.0,0.179995
1,1,1,participant1,"Thats good for you, i'm not very into new tech",Human,0xab38710,"['s', 'good', ',', 'new', 'tech']","love','iphone','!','bought','new','s','good','','tech','college','student','gym','live','donations','vegan','midwest','...','dogs','maybe','told','eat','cheap','salads','insted','meat','mind','having','office','hard','Dogs','?','vegetarian','Strange','answer','guess','know','ps4','experience','.","s','good','','new','tech','gym','live','donations','vegan','...','dogs','maybe','told','eat','cheap','salads','insted','meat','Dogs','office','?','Strange','answer","love','iphone','!','bought','new','college','student','vegan','midwest','mind','having','office','hard','vegetarian','guess','know','ps4','experience','.","vegan','office','new",6.0,39.0,23.0,19.0,3.0,0.153846,0.260870,0.052632,0.130435,0.589744,46.0,10.0,10.0,11.0,0.183899
2,2,2,participant2,I am a college student and i am a college student,Bot,0xab38710,"['college', 'student', 'college', 'student']","love','iphone','!','bought','new','s','good','','tech','college','student','gym','live','donations','vegan','midwest','...','dogs','maybe','told','eat','cheap','salads','insted','meat','mind','having','office','hard','Dogs','?','vegetarian','Strange','answer','guess','know','ps4','experience','.","s','good','','new','tech','gym','live','donations','vegan','...','dogs','maybe','told','eat','cheap','salads','insted','meat','Dogs','office','?','Strange','answer","love','iphone','!','bought','new','college','student','vegan','midwest','mind','having','office','hard','vegetarian','guess','know','ps4','experience','.","vegan','office','new",4.0,39.0,23.0,19.0,3.0,0.102564,0.210526,0.000000,0.157895,0.487179,49.0,11.0,7.0,10.0,0.107382
3,3,3,participant1,I am go to gym and live on donations,Human,0xab38710,"['gym', 'live', 'donations']","love','iphone','!','bought','new','s','good','','tech','college','student','gym','live','donations','vegan','midwest','...','dogs','maybe','told','eat','cheap','salads','insted','meat','mind','having','office','hard','Dogs','?','vegetarian','Strange','answer','guess','know','ps4','experience','.","s','good','','new','tech','gym','live','donations','vegan','...','dogs','maybe','told','eat','cheap','salads','insted','meat','Dogs','office','?','Strange','answer","love','iphone','!','bought','new','college','student','vegan','midwest','mind','having','office','hard','vegetarian','guess','know','ps4','experience','.","vegan','office','new",3.0,39.0,23.0,19.0,3.0,0.076923,0.130435,0.000000,0.130435,0.589744,36.0,9.0,9.0,8.0,0.109968
4,4,4,participant2,I am a vegan and i am in the 

In [ ]:
# Import Drive API and authenticate.
from google.colab import drive

# Mount your Drive to the Colab VM.
drive.mount('/gdrive')

# Write the DataFrame to CSV file.
df_diccs_parametros_uniqWeight.to_csv('/gdrive/My Drive/df_diccs_parametros_uniqW.csv')


Mounted at /gdrive


NameError: ignored

In [ ]:
#para levantarlo de drive si luego necesitan modificarlo sin procesar todo lo anterior
df_diccs_parametros_uniqW=pd.read_csv('/gdrive/My Drive/df_diccs_parametros_uniqW.csv')  

In [ ]:
df_diccs_parametros_uniqW.head()

,Unnamed: 0,Unnamed: 0.1,id,sender,text,sender_class,dialog_id,filtered_tokens_col,diccionario_total,diccionario_participant1,diccionario_participant2,diccionario_interseccion,cant_key_words,cant_diccionario_total,cant_diccionario_participant1,cant_diccionario_participant2,cant_diccionario_interseccion,keyWords_over_totalDic,keyWords_over_senderDic,keyWords_over_listenerDic,intersect_over_senderDic,participantdicc_over_totaldicc,cant_caracteres,cant_palabras_interaccion,cant_palabras_unicas,cant_puntuacion,uniqWeight
0,0,0,0,participant2,I love iphone! i just bought new iphone!,Bot,0xab38710,"['love', 'iphone', '!', 'bought', 'new', 'ipho...","love','iphone','!','bought','new','s','good','...","s','good','','new','tech','gym','live','donati...","love','iphone','!','bought','new','college','s...","vegan','office','new",7.0,39.0,23.0,19.0,3.0,0.179487,0.368421,0.043478,0.157895,0.487179,40.0,8.0,7.0,9.0,0.179995
1,1,1,1,participant1,"Thats good for you, i'm not very into new tech",Human,0xab38710,"['s', 'good', ',', 'new', 'tech']","love','iphone','!','bought','new','s','good','...","s','good','','new','tech','gym','live','donati...","love','iphone','!','bought','new','college','s...","vegan','office','new",6.0,39.0,23.0,19.0,3.0,0.153846,0.260870,0.052632,0.130435,0.589744,46.0,10.0,10.0,11.0,0.183899
2,2,2,2,participant2,I am a college student and i am a college student,Bot,0xab38710,"['college', 'student', 'college', 'student']","love','iphone','!','bought','new','s','good','...","s','good','','new','tech','gym','live','donati...","love','iphone','!','bought','new','college','s...","vegan','office','new",4.0,39.0,23.0,19.0,3.0,0.102564,0.210526,0.000000,0.157895,0.487179,49.0,11.0,7.0,10.0,0.107382
3,3,3,3,participant1,I am go to gym and live on donations,Human,0xab38710,"['gym', 'live', 'donations']","love','iphone','!','bought','new','s','good','...","s','good','','new','tech','gym','live','donati...","love','iphone','!','bought','new','college','s...","vegan','office','new",3.0,39.0,23.0,19.0,3.0,0.076923,0.130435,0.000000,0.130435,0.589744,36.0,9.0,9.0,8.0,0.109968
4,4,4,4,participant2,I am a vegan and i am in the midwest,Bot,0xab38710,"['vegan', 'midwest']","love','iphone','!','bought','new','s','good','...","s','good','','new','tech','gym','live','donati...","love','iphone','!','bought','new','college','s...","vegan','office','new",2.0,39.0,23.0,19.0,3.0,0.051282,0.105263,0.043478,0.157895,0.487179,36.0,10.0,9.0,9.0,0.152454


In [ ]:
#aca podemos cambiar el uniqWeight
df_diccs_parametros_uniqW['uniqWeight']=df_diccs_parametros_uniqW.apply(lambda row: row.keyWords_over_listenerDic+((row.cant_diccionario_interseccion/row.cant_diccionario_total)*(0.25*row.keyWords_over_totalDic + 0.25*row.keyWords_over_senderDic + 2*row.keyWords_over_listenerDic + (row.cant_palabras_interaccion + row.cant_palabras_unicas + row.cant_palabras_interaccion)/(3*row.cant_palabras_interaccion) + (row.cant_puntuacion/row.cant_caracteres))), axis=1)

In [ ]:
df_diccs_parametros_uniqW.head()

,Unnamed: 0,Unnamed: 0.1,id,sender,text,sender_class,dialog_id,filtered_tokens_col,diccionario_total,diccionario_participant1,diccionario_participant2,diccionario_interseccion,cant_key_words,cant_diccionario_total,cant_diccionario_participant1,cant_diccionario_participant2,cant_diccionario_interseccion,keyWords_over_totalDic,keyWords_over_senderDic,keyWords_over_listenerDic,intersect_over_senderDic,participantdicc_over_totaldicc,cant_caracteres,cant_palabras_interaccion,cant_palabras_unicas,cant_puntuacion,uniqWeight
0,0,0,0,participant2,I love iphone! i just bought new iphone!,Bot,0xab38710,"['love', 'iphone', '!', 'bought', 'new', 'ipho...","love','iphone','!','bought','new','s','good','...","s','good','','new','tech','gym','live','donati...","love','iphone','!','bought','new','college','s...","vegan','office','new",7.0,39.0,23.0,19.0,3.0,0.179487,0.368421,0.043478,0.157895,0.487179,40.0,8.0,7.0,9.0,0.151730
1,1,1,1,participant1,"Thats good for you, i'm not very into new tech",Human,0xab38710,"['s', 'good', ',', 'new', 'tech']","love','iphone','!','bought','new','s','good','...","s','good','','new','tech','gym','live','donati...","love','iphone','!','bought','new','college','s...","vegan','office','new",6.0,39.0,23.0,19.0,3.0,0.153846,0.260870,0.052632,0.130435,0.589744,46.0,10.0,10.0,11.0,0.164022
2,2,2,2,participant2,I am a college student and i am a college student,Bot,0xab38710,"['college', 'student', 'college', 'student']","love','iphone','!','bought','new','s','good','...","s','good','','new','tech','gym','live','donati...","love','iphone','!','bought','new','college','s...","vegan','office','new",4.0,39.0,23.0,19.0,3.0,0.102564,0.210526,0.000000,0.157895,0.487179,49.0,11.0,7.0,10.0,0.089319
3,3,3,3,participant1,I am go to gym and live on donations,Human,0xab38710,"['gym', 'live', 'donations']","love','iphone','!','bought','new','s','good','...","s','good','','new','tech','gym','live','donati...","love','iphone','!','bought','new','college','s...","vegan','office','new",3.0,39.0,23.0,19.0,3.0,0.076923,0.130435,0.000000,0.130435,0.589744,36.0,9.0,9.0,8.0,0.098005
4,4,4,4,participant2,I am a vegan and i am in the midwest,Bot,0xab38710,"['vegan', 'midwest']","love','iphone','!','bought','new','s','good','...","s','good','','new','tech','gym','live','donati...","love','iphone','!','bought','new','college','s...","vegan','office','new",2.0,39.0,23.0,19.0,3.0,0.051282,0.105263,0.043478,0.157895,0.487179,36.0,10.0,9.0,9.0,0.146767


In [ ]:
# Import Drive API and authenticate.
from google.colab import drive

# Mount your Drive to the Colab VM.
drive.mount('/gdrive')

# Write the DataFrame to CSV file.
df_diccs_parametros_uniqW.to_csv('/gdrive/My Drive/df_diccs_parametros_uniqW03102021.csv')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
